In [1]:
%%javascript
$.getScript('http://asimjalis.github.io/ipyn-ext/js/ipyn-present.js')

<IPython.core.display.Javascript object>

<h1 class="tocheading">Serving Layer</h1>

> The serving layer consists of databases that index and serve the results of the batch layer.

<div id="toc"></div>

Objectives
---------------------------------------------------------------------
By the end of today's lesson, you will be able to:  
* Tailor batch views to the queries they serve
* Provide a new answer to the data-normalization versus
denormalization debate
* Discuss the advantages of batch-writable, random-read, and no random-write databases
* Contrast a Lambda Architecture solution with a fully incremental solution

Computing on the batch layer
---------------------------------------------------------------------
**The serving layer provides low-latency access to the results of calculations performed on the master dataset.**  
The serving layer views are slightly out of date due to the time required for batch computation.
![Figure 10.1](images/10fig01_alt.jpg)

While investigating the serving layer, you’ll learn the following:

* Indexing strategies to minimize latency, resource usage, and variance
* The requirements for the serving layer in the Lambda Architecture
* How the serving layer solves the long-debated normalization versus denormal-
ization problem

## Performance metrics for the serving layer

* _latency_ - the time required to answer a single query
* _throughput_ - the number of queries that can be served within a given period of time

### Scatter/Gather

Suppose a query requires fetching data from three servers...
![Figure 10.3](images/10fig03_alt.jpg)
**When distributing a task over multiple servers, the overall latency is determined by the slowest server response time.**

In general, the more servers a query touches, the higher the overall latency of the query.  
![Figure 10.4](images/10fig04_alt.jpg)
**If you increase the number of servers involved in a distributed task, you also increase the likelihood that at least one will respond slowly.**

### A different indexing strategy
Collocate the pageview information for a sin- gle URL on the same partition and store it sequentially. Fetching the pageviews will then only require a single seek and scan rather than numerous seeks.  
![Figure 10.5](images/10fig05_alt.jpg)
**A sorted index promotes scans and limits disk seeks to improve both latency and throughput.**

## The serving layer solution to the normalization/ denormalization problem
### A normalized schema uses multiple independent datasets with little or no redundant data.

| User ID | Name | Location ID |
|:-:| ------ |:-:|
| 1 | Sally  | 3 |
| 2 | George | 1 |
| 3 | Bob    | 3 |

| Location ID | City | State | Population |
|:------:| ---- | -- | ----:|
| 1 | New York  | NY | 8.2M |
| 2 | San Diego | CA | 1.3M |
| 3 | Chicago   | IL | 2.7M |

### Denormalized tables store data redundantly to improve query performance.

| User ID | Name | Location ID | City | State |
|:-----:| ------ |:--:| ---- | -- |
| 1 | Sally  | 3 | Chicago   | IL | 
| 2 | George | 1 | New York  | NY |
| 3 | Bob    | 3 | Chicago   | IL | 

| Location ID | City | State | Population |
|:------:| ---- | -- | ----:|
| 1 | New York  | NY | 8.2M |
| 2 | San Diego | CA | 1.3M |
| 3 | Chicago   | IL | 2.7M |

The master dataset should be normalized to maintain consistency but the batch views may be denormalized for query efficiency. Since the batch views are _views_, this does not violate the principles of normaliziation.

## Requirements for a serving layer database
* *Batch writable*—The batch views for a serving layer are produced from scratch. When a new version of a view becomes available, it must be possible to completely swap out the older version with the updated view.
* *Scalable*—A serving layer database must be capable of handling views of arbitrary size. As with the distributed filesystems and batch computation framework previously discussed, this requires it to be distributed across multiple machines.
* *Random reads*—A serving layer database must support random reads, with indexes providing direct access to small portions of the view. This requirement is necessary to have low latency on queries.
* *Fault-tolerant*—Because a serving layer database is distributed, it must be tolerant of machine failures.

## Designing a serving layer for SuperWebAnalytics.com

In [2]:
import pyspark
import simplejson as json

from pyspark.sql.types import *
from pyspark.sql.functions import approxCountDistinct

try:
    if not sc:
        sc = pyspark.SparkContext()
except NameError:
    sc = pyspark.SparkContext()

sqlContext = pyspark.HiveContext(sc)

In [3]:
distinct_normalized_page_views = sc.pickleFile("../SuperWebAnalytics/batch/distinct_normalized_page_views/")

### Pageviews over time

In [4]:
granularity = {'h': 60 * 60}
granularity['d'] = granularity['h'] * 24 # days in UTC (GMT) timezone
granularity['w'] = granularity['d'] * 7
granularity['m'] = granularity['w'] * 4  # 1 month = 28 days, not 1 calendar month
granularity

{'d': 86400, 'h': 3600, 'm': 2419200, 'w': 604800}

In [5]:
def gen_granular(granual):
    def func(datum): 
        url = datum['dataunit']['page_view']['page']['url']
        true_as_of_secs = datum['pedigree']['true_as_of_secs']
        return (url, granual, true_as_of_secs - true_as_of_secs % granularity[granual])
    return func

In [6]:
page_view_schema = StructType([
    StructField('url', StringType() ,True),
    StructField('granularity', StringType(), True),
    StructField('ts', IntegerType(), True)
    ])

In [7]:
page_view_by_hour = sqlContext.createDataFrame(distinct_normalized_page_views.map(gen_granular('h')), 
                                               page_view_schema)

In [8]:
hourly_page_views = page_view_by_hour.groupBy(['url', 'granularity', 'ts']).count()

In [9]:
hourly_page_views.registerTempTable('hourly_page_views')
sqlContext.cacheTable('hourly_page_views')

# N.B. This won't work if you want your days to be in any timezone other than UTC
daily_page_views = sqlContext.sql("""SELECT url, 'd' AS granularity, ts - ts % {} AS ts, sum(count) AS count 
                                FROM hourly_page_views 
                                GROUP BY url, ts - ts % {}""".format(granularity['d'], granularity['d']))

In [10]:
page_view_union = hourly_page_views.unionAll(daily_page_views).orderBy('url')
page_view_union.show()

+---------------+-----------+----------+------+
|            url|granularity|        ts| count|
+---------------+-----------+----------+------+
|    mysite.com/|          h|1438912800|300054|
|    mysite.com/|          h|1438624800|101212|
|    mysite.com/|          d|1438905600|300054|
|    mysite.com/|          d|1438560000|101212|
|mysite.com/blog|          h|1438912800|899778|
|mysite.com/blog|          h|1438624800|101732|
|mysite.com/blog|          d|1438560000|101732|
|mysite.com/blog|          d|1438905600|899778|
+---------------+-----------+----------+------+



In [11]:
page_view_union.write.parquet("../SuperWebAnalytics/batch_views/page_views", mode='overwrite')

### Uniques over time

![Figure 10.8](images/10fig08_alt.jpg)

**Index design for uniques over time. Although the index keys are a compound of URL and granularity, indexes are partitioned between servers solely by the URL.**

* key is a compound key of URL and granularity
* indices are partitioned solely by  URL  

To retrieve a range of values for a URL and granularity, you’d use the URL to find the server containing the information you need, and then use both the URL and granularity to look up the values you’re interested in.

In [12]:
def user_row(granual):
    import simplejson as json
    granularity = {'h': 60 * 60}
    granularity['d'] = granularity['h'] * 24 # days in UTC (GMT) timezone
    granularity['w'] = granularity['d'] * 7
    granularity['m'] = granularity['w'] * 4  # 1 month = 28 days, not 1 calendar month
    def func(datum):
        url = datum['dataunit']['page_view']['page']['url']
        true_as_of_secs = datum['pedigree']['true_as_of_secs']
        person = json.dumps(datum['dataunit']['page_view']['person'])
        return (url, granual, true_as_of_secs - true_as_of_secs % granularity[granual], person)
    return func

visit_schema = StructType([
    StructField('url', StringType() ,True),
    StructField('granularity', StringType(), True),
    StructField('ts', IntegerType(), True),
    StructField('person', StringType(), True)
    ])

In [13]:
hourly_visits = sqlContext.createDataFrame(distinct_normalized_page_views.map(user_row('h')),
                                           visit_schema)
hourly_visits.head(5)

[Row(url=u'mysite.com/blog', granularity=u'h', ts=1438912800, person=u'{"cookie": "PQRST"}'),
 Row(url=u'mysite.com/blog', granularity=u'h', ts=1438912800, person=u'{"user_id": 5693}'),
 Row(url=u'mysite.com/blog', granularity=u'h', ts=1438912800, person=u'{"user_id": 6257}'),
 Row(url=u'mysite.com/', granularity=u'h', ts=1438912800, person=u'{"user_id": 765}'),
 Row(url=u'mysite.com/blog', granularity=u'h', ts=1438912800, person=u'{"user_id": 7132}')]

In [15]:
hourly_visits.write.parquet('../SuperWebAnalytics/batch/hourly_visits/', mode='overwrite')

In [16]:
visits_by_hour = hourly_visits.groupBy(['url', 'granularity', 'ts'])\
                              .agg(approxCountDistinct('person').alias('approx_uniques'))

In [17]:
daily_visits = sqlContext.createDataFrame(distinct_normalized_page_views.map(user_row('d')),
                                           visit_schema)
daily_visits.write.parquet('../SuperWebAnalytics/batch/daily_visits/', mode='overwrite')

In [18]:
visits_by_day = daily_visits.groupBy(['url', 'granularity', 'ts'])\
                              .agg(approxCountDistinct('person').alias('approx_uniques'))

In [19]:
visits_union = visits_by_hour.unionAll(visits_by_day).orderBy('url')
visits_union.show()

+---------------+-----------+----------+--------------+
|            url|granularity|        ts|approx_uniques|
+---------------+-----------+----------+--------------+
|    mysite.com/|          d|1438560000|         10874|
|    mysite.com/|          d|1438905600|         10893|
|    mysite.com/|          h|1438624800|         10874|
|    mysite.com/|          h|1438912800|         10893|
|mysite.com/blog|          h|1438912800|         10893|
|mysite.com/blog|          h|1438624800|         10776|
|mysite.com/blog|          d|1438560000|         10776|
|mysite.com/blog|          d|1438905600|         10893|
+---------------+-----------+----------+--------------+



In [20]:
visits_union.write.parquet("../SuperWebAnalytics/batch_views/visits", mode='overwrite')

#### User-identifier normalization
![Figure 9.3](images/09fig03_alt.jpg)

<!--
**Equivs could affect any bucket in the database.**
![Figure 10.10](images/10fig10.jpg)
-->

#### Handling equivs on the read-side workflow
![Figure 10.11](images/10fig11_alt.jpg)

1. First, retrieve every UserID set for every hour in the range, and merge them.
2. Convert the set of UserIDs to a set of PersonIDs by using the UserID-to-PersonID index.
3. Return the count of the PersonID set.

<!--
### Bounce-rate analysis

![Figure 10.9](images/10fig09_alt.jpg)

**Implementing a bounce-rates view using a key/value index**
-->

In [21]:
visits = sqlContext.read.parquet("../SuperWebAnalytics/batch_views/visits")
visits.registerTempTable("visits");
sqlContext.sql("""SELECT from_unixtime(ts) AS dt, approx_uniques FROM visits 
                   WHERE url = 'mysite.com/' AND granularity = 'd'""").show()

+-------------------+--------------+
|                 dt|approx_uniques|
+-------------------+--------------+
|2015-08-02 17:00:00|         10874|
|2015-08-06 17:00:00|         10893|
+-------------------+--------------+



# Lab

By this coming Monday:  

1. Create batch views in tabular (not key-value) format.
2. Serialize them using parquet.
3. Load them and query them using SQL (_i.e._ HiveQL or SparkSQL)